In [1]:
# # An architecture similar to LeNet, except that the list of *all* actions comes in at the first fully-connected layer
# # Useful as a discriminator for GAIL-like methods

# class Net(nn.Module):
    
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(5, 6, 5, padding=2)
#         self.pool1 = nn.AvgPool2d(2, stride=2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.pool2 = nn.AvgPool2d(2, stride=2)
#         self.fc1 = nn.Linear(576+10, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 1)
    
#     def forward(self, x0, x1, x2, x3, x4, a0, a1, a2, a3, a4):
#         acts = torch.cat((a0, a1, a2, a3, a4), dim=1)
#         z = torch.stack((x0, x1, x2, x3, x4), dim=1)
#         z = F.relu(self.conv1(z))
#         z = self.pool1(z)
#         z = F.relu(self.conv2(z))
#         z = self.pool2(z)
#         z = F.relu(self.fc1(torch.cat((torch.flatten(z, 1), acts), dim=1)))
#         z = F.relu(self.fc2(z))
#         # z = self.fc3(z)
#         return self.fc3(z), z

#     def forward_stacked(self, img, act, exp):
#         return self.forward(img[:,0,:,:], img[:,1,:,:], img[:,2,:,:], img[:,3,:,:], img[:,4,:,:], act[:,0,:], act[:,1,:], act[:,2,:], act[:,3,:], exp)

# discrim = Net()
# loss = F.mse_loss
# sgd = opt.SGD(discrim.parameters(), lr=0.001)

In [ ]:
# def train_aux(dl, net, loss, opt, epochs=1):
#     n = len(dl.dataset)
#     ttl_err = 0
#     ttl_ex = 0
#     for ep in range(epochs):
#         for _, (img, act, exp, aux) in enumerate(dl):
#             yhat, _ = net.forward_stacked(img, act, exp)
#             err = loss(yhat, aux)
#             opt.zero_grad()
#             err.backward()
#             opt.step()
#             ttl_err += err.item() * img.shape[0]
#             ttl_ex += img.shape[0]
#         print(f"epoch: {ep:>5d}  loss: {ttl_err/ttl_ex:>7f}  processed: {ttl_ex:>5d}/{n}")

# def test_aux(dl, net, loss):
#     with torch.no_grad():
#         ttl_loss = 0
#         ttl_ex = 0
#         for _, (img, act, exp, aux) in enumerate(dl):
#             yhat, _ = net.forward_stacked(img, act, exp)
#             err = loss(yhat, aux).item()
#             ttl_loss += err * img.shape[0]
#             ttl_ex += img.shape[0]
#         avg_loss = ttl_loss/ttl_ex
#         print(f"loss: {avg_loss:>7f}")
#         return avg_loss

# def scatter_aux(dl, net, jitter = 0.05):
#     with torch.no_grad():
#         ys = []
#         yhats = []
#         for _, (img, act, exp, aux) in enumerate(dl):
#             yhat, _ = net.forward_stacked(img, act, exp)
#             ys.append(aux)
#             yhats.append(yhat)
#         ys = torch.cat(ys)
#         yhats = torch.cat(yhats)
#         if jitter:
#             ys += jitter * torch.randn(ys.shape)
#         fig = plt.plot(ys[:,0], yhats[:,0], '.')
#         plt.ylabel('predicted')
#         plt.xlabel('actual')
#         # plt.legend(['x', 'y'])
#         return fig


In [1]:
# scatter_aux(discrim_data, discrim, jitter=0.3)
# test_aux(discrim_data, discrim, loss)
# train_aux(discrim_data, discrim, loss, sgd, epochs=1000)

In [ ]:
# hires = DrivingDataset('data/hires.json.gz', name='hires')
# hires_dl = td.DataLoader(hires, batch_size = 64)

td1env = DrivingDataset('data/assist-bc-noise-2.json', name='td1env', window=6)
print(td1env)
td2env = DrivingDataset('data/inject-noise.json', name='td2env', window=6)
print(td2env)
td3env = DrivingDataset('data/assist-dagger-fix.json', name='td3env', window=6)
print(td3env)
td4env = DrivingDataset('data/assist-dagger-fix-2.json', name='td4env', window=6)
print(td4env)
tstenv = DrivingDataset('data/driving-data-w-acts-ex.json', name='tstenv', window=6)
print(tstenv)
train_env = td.DataLoader(td.ConcatDataset([td1env, td2env, td3env, td4env]), batch_size = 64)
test_env = td.DataLoader(tstenv, batch_size = 64)

In [2]:
# An architecture for learning an environment model: state encoder is like a LeNet
# Action encoder is a small fully-connected network
# Transition model is linear in state \otimes action, followed by normalization
# Decoder is like a reversed LeNet, shared for observations at t and t+1

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        state_dim = 84
        self.fc1_act = nn.Linear(2, state_dim)
        self.fc2_act = nn.Linear(state_dim, state_dim)
        self.trans = nn.Linear(state_dim, state_dim)
        self.conv1 = nn.Conv2d(4, 6, 5, padding=2)
        self.pool1 = nn.AvgPool2d(2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.AvgPool2d(2, stride=2)
        self.fc1 = nn.Linear(576+8, 120)
        # self.fc1b = nn.Linear(576+10, 120)
        self.fc2 = nn.Linear(120, state_dim)
        # self.fc2b = nn.Linear(120+2, state_dim)
        self.fc3 = nn.Linear(state_dim, state_dim)
        self.fc4 = nn.Linear(state_dim, 1024)
        # self.deconv1 = nn.ConvTranspose2d(16, 6, 5, stride=2)
        # self.deconv2 = nn.ConvTranspose2d(6, 1, 5, stride=2, padding=1, output_padding=1)

        # self.fc3 = nn.Linear(state_dim, 120)
        # self.fc4 = nn.Linear(120, 576)

    def load_onnx(self, model):
        wts = { t.name: torch.tensor(onnx.numpy_helper.to_array(t)) for t in model.graph.initializer }
        self.load_state_dict(wts)

    def encode_act(self, act):
        return self.fc2_act(F.relu(self.fc1_act(act)))

    def transition(self, current, act):
        return self.trans(current)
        # return self.trans(current * self.encode_act(act))
        

    def forward(self, x0, x1, x2, x3, a0, a1, a2, a3, a4):
        acts = torch.cat((a0, a1, a2, a3), dim=1)
        z = torch.stack((x0, x1, x2, x3), dim=1)

        # two convolutional layers
        z = F.relu(self.conv1(z))
        z = self.pool1(z)
        z = F.relu(self.conv2(z))
        z = self.pool2(z)

        # cat in the past actions, apply two fully connected layers to get the embedding
        # for the current state
        z = torch.flatten(z, 1)
        embed_a = F.relu(self.fc1(torch.cat((z, acts), dim=1)))
        embed_a = F.relu(self.fc2(embed_a))

        # apply transition function to get embedding for next state
        embed_b = self.transition(embed_a, a4)

        # # cat in the past actions, apply two fully connected layers to get each embedding
        # # note the second embedding uses one extra action
        # z = torch.flatten(z, 1)
        # embed_a = F.relu(self.fc1a(torch.cat((z, acts), dim=1)))
        # embed_a = F.relu(self.fc2a(embed_a))
        # embed_b = F.relu(self.fc1b(torch.cat((z, acts, a5), dim=1)))
        # embed_b = F.relu(self.fc2b(embed_b))

        # decode the first output
        za = F.relu(self.fc3(embed_a))
        za = F.relu(self.fc4(za)).reshape((-1, 32, 32))
        # za = F.relu(self.fc4(za)).reshape((-1, 16, 6, 6))
        # za = F.relu(self.deconv1(za))
        # za = F.relu(self.deconv2(za)).squeeze(dim=1)

        # decode the second output
        zb = F.relu(self.fc3(embed_b))
        zb = F.relu(self.fc4(zb)).reshape((-1, 32, 32))
        # zb = F.relu(self.fc4(zb)).reshape((-1, 16, 6, 6))
        # zb = F.relu(self.deconv1(zb))
        # zb = F.relu(self.deconv2(zb)).squeeze(dim=1)

        # return both outputs and both embeddings
        return za, zb, embed_a, embed_b

    def forward_stacked(self, img, act):
        return self.forward(img[:,0,:,:], img[:,1,:,:], img[:,2,:,:], img[:,3,:,:], act[:,0,:], act[:,1,:], act[:,2,:], act[:,3,:], act[:,4,:])

net = Net()
loss = F.mse_loss
sgd = opt.SGD(net.parameters(), lr=0.001)

NameError: name 'nn' is not defined

In [ ]:
def train_emodel(dl, net, loss, opt, epochs=1):
    n = len(dl.dataset)
    ttl_err = 0
    ttl_ex = 0
    for ep in range(epochs):
        for _, (img, act, _, _, _) in enumerate(dl):
            yhat1, yhat2, _, _ = net.forward_stacked(img[:,:-2,:,:], act)
            err = loss(yhat1, img[:,-2,:,:])
            err += loss(yhat2, img[:,-1,:,:])
            opt.zero_grad()
            err.backward()
            opt.step()
            ttl_err += err.item() * img.shape[0]
            ttl_ex += img.shape[0]
        print(f"epoch: {ep:>5d}  loss: {ttl_err/ttl_ex:>7f}  processed: {ttl_ex:>5d}/{n}")

def test_emodel(dl, net, loss):
    n = len(dl.dataset)
    ttl_err = 0
    ttl_ex = 0
    for _, (img, act, _, _, _) in enumerate(dl):
        yhat1, yhat2, _, _ = net.forward_stacked(img[:,:-2,:,:], act)
        err = loss(yhat1, img[:,-2,:,:])
        err += loss(yhat2, img[:,-1,:,:])
        ttl_err += err.item() * img.shape[0]
        ttl_ex += img.shape[0]
    print(f"loss: {ttl_err/ttl_ex:>7f} on {ttl_ex:>5d} examples")

In [ ]:
train_emodel(train_env, net, loss, sgd, epochs=10)
test_emodel(test_env, net, loss)
yhat1, yhat2, _, _ = net.forward_stacked(img[:,:-2,:,:], act)
show_sa(yhat1[t,:,:].detach(), act[t,4,:])

In [ ]:
steps = 31
randfeat = 100
xs = torch.linspace(-1, 1, steps)
gx, gy = torch.meshgrid(xs, xs)
Wx = torch.randn(randfeat)*5
Wy = torch.randn(randfeat)*5
b = torch.rand(randfeat) * 2 * math.pi
x, y = (.3, -.4)
embed = torch.sin(Wx * x + Wy * y + b)
embeds = torch.sin(Wx.reshape((-1, 1, 1)) * gx + Wy.reshape((-1, 1, 1)) * gy + b.reshape(-1, 1, 1))
gz = torch.einsum('i,ijk', embed, embeds)/randfeat
dd = max(xs) + 0.5 * (xs[-1] - xs[-2])
plt.imshow(gz.t(), extent = (-dd, dd, -dd, dd), origin='lower')
plt.plot(x, y, 'x')
plt.colorbar()